<a href="https://colab.research.google.com/github/dimakpa/DL_Projects/blob/main/clusterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import matplotlib as plt
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import pyplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

# df = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/MEPHI/Cluster/pr1.csv"))
df = pd.DataFrame(pd.read_excel("/content/drive/MyDrive/Colab Notebooks/MEPHI/Cluster/primer.xlsx"))

df.head()

,id,component,action,target,objecttable,objectid,userid,courseid,relateduserid,ip,timecreated,origin
0,357694488,core,graded,user,grade_grades,33859184.0,42067,8580,27928.0,77.73.31.51,1681309797,web
1,357694489,mod_assign,graded,submission,assign_grades,1059298.0,42067,8580,27928.0,77.73.31.51,1681309797,web
2,357694490,mod_quiz,viewed,attempt,quiz_attempts,1842831.0,64913,4436,64913.0,45.147.80.190,1681309797,web
3,357694491,core,viewed,course,NaN,NaN,66783,2616,NaN,213.87.158.247,1681309797,web
4,357694492,mod_assign,viewed,grading_form,NaN,NaN,42067,8580,25912.0,77.73.31.51,1681309797,web


Проанализируем данные

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47375 entries, 0 to 47374
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             47375 non-null  int64  
 1   component      47375 non-null  object 
 2   action         47375 non-null  object 
 3   target         47375 non-null  object 
 4   objecttable    24829 non-null  object 
 5   objectid       24829 non-null  float64
 6   userid         47375 non-null  int64  
 7   courseid       47375 non-null  int64  
 8   relateduserid  15260 non-null  float64
 9   ip             46732 non-null  object 
 10  timecreated    47375 non-null  int64  
 11  origin         47375 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 4.3+ MB


In [ ]:
df.shape

(47375, 12)

Посмотрим на пропущенные значения

In [ ]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

In [ ]:
miss_values = missing_values_table(df)
miss_values

Your selected dataframe has 12 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
relateduserid,32115,67.8
objecttable,22546,47.6
objectid,22546,47.6
ip,643,1.4


3 столбца содержат очень много пропусков около 50% и они уникальны для каждого (id) -> можем просто удалить их, тк для кластеризации они нам никак не нужны

id тоже не нужен, он уникальный

##Если нужно предсказать origin, то этот столбец тоже удалим из датасета и проведем LabelEncoding для проверки (ЭТО ПОХОДУ НЕ НАДО)

In [ ]:
df1 = pd.DataFrame()
df1 = df.drop(['relateduserid', 'objecttable', 'objectid', 'id'], axis=1).copy()
df1.head()

,component,action,target,userid,courseid,ip,timecreated,origin
0,core,graded,user,42067,8580,77.73.31.51,1681309797,web
1,mod_assign,graded,submission,42067,8580,77.73.31.51,1681309797,web
2,mod_quiz,viewed,attempt,64913,4436,45.147.80.190,1681309797,web
3,core,viewed,course,66783,2616,213.87.158.247,1681309797,web
4,mod_assign,viewed,grading_form,42067,8580,77.73.31.51,1681309797,web


Удалим строки, где нет ip

In [ ]:
df1 = df1.dropna()

In [ ]:
miss_values = missing_values_table(df1)
miss_values

Your selected dataframe has 8 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


Посмотрим сколько уникальных категорий в каждом столбце


In [ ]:
df1['component'].value_counts()

core                           24600
mod_assign                     11042
mod_quiz                        5622
mod_resource                    2986
assignsubmission_file            940
mod_folder                       570
mod_forum                        218
mod_page                         152
mod_url                          141
gradereport_overview             116
mod_choice                        87
gradereport_user                  70
gradereport_grader                58
mod_choicegroup                   36
mod_book                          29
assignsubmission_comments         19
gradereport_singleview            18
report_log                        11
assignsubmission_onlinetext       10
mod_data                           3
mod_chat                           2
mod_feedback                       1
report_outline                     1
Name: component, dtype: int64

Удалим значения, где уникальных очень мало (меньше 100)

In [ ]:
df1 = df1[df1['component'] != 'mod_choice']
df1 = df1[df1['component'] != 'gradereport_user']
df1 = df1[df1['component'] != 'gradereport_grader']
df1 = df1[df1['component'] != 'mod_choicegroup']
df1 = df1[df1['component'] != 'mod_book']
df1 = df1[df1['component'] != 'tool_recyclebin']
df1 = df1[df1['component'] != 'assignsubmission_comments']
df1 = df1[df1['component'] != 'gradereport_singleview']
df1 = df1[df1['component'] != 'report_log']
df1 = df1[df1['component'] != 'assignsubmission_onlinetext']
df1 = df1[df1['component'] != 'mod_data']
df1 = df1[df1['component'] != 'mod_chat']
df1 = df1[df1['component'] != 'mod_feedback']
df1 = df1[df1['component'] != 'report_outline']
df1['component'].value_counts()

core                     24600
mod_assign               11042
mod_quiz                  5622
mod_resource              2986
assignsubmission_file      940
mod_folder                 570
mod_forum                  218
mod_page                   152
mod_url                    141
gradereport_overview       116
Name: component, dtype: int64

In [ ]:
df1['action'].value_counts()

viewed        34684
loggedin       3144
failed         2626
updated        1600
graded         1139
submitted       623
sent            480
uploaded        472
created         444
called          394
loggedout       177
started         142
reviewed        127
added            68
searched         65
locked           62
deleted          62
assigned         41
removed          16
downloaded        7
unlocked          5
unassigned        5
granted           2
accepted          1
reset             1
Name: action, dtype: int64

In [ ]:
df1 = df1[df1['action'] != 'added']
df1 = df1[df1['action'] != 'searched']
df1 = df1[df1['action'] != 'locked']
df1 = df1[df1['action'] != 'assigned']
df1 = df1[df1['action'] != 'removed']
df1 = df1[df1['action'] != 'unassigned']
df1 = df1[df1['action'] != 'downloaded']
df1 = df1[df1['action'] != 'unlocked']
df1 = df1[df1['action'] != 'granted']
df1 = df1[df1['action'] != 'accepted']
df1 = df1[df1['action'] != 'reset']
df1['action'].value_counts()

viewed       34684
loggedin      3144
failed        2626
updated       1600
graded        1139
submitted      623
sent           480
uploaded       472
created        444
called         394
loggedout      177
started        142
reviewed       127
deleted         62
Name: action, dtype: int64

In [ ]:
df1.shape

(46114, 8)

In [ ]:
len(df1['timecreated'].unique())

12322

In [ ]:
# df1['origin'].value_counts()

In [ ]:
df1['ip'].value_counts()

193.41.142.196    2587
193.41.142.136    1486
193.41.142.36     1301
45.147.80.190     1047
77.73.31.51        394
                  ... 
194.28.28.97         1
83.220.237.127       1
178.57.48.18         1
46.138.154.89        1
85.249.42.51         1
Name: ip, Length: 2863, dtype: int64

In [ ]:
df1.tail()

,component,action,target,userid,courseid,ip,timecreated,origin
47370,core,loggedin,user,66506,0,212.16.10.193,1681323726,web
47371,core,viewed,dashboard,66506,0,212.16.10.193,1681323727,web
47372,mod_resource,viewed,course_module,20449,9292,195.225.109.79,1681323727,web
47373,core,viewed,course,58446,2182,109.252.168.38,1681323728,web
47374,core,failed,webservice_login,0,0,193.41.142.196,1681323728,ws


timecreated колонка вероятно отвечает за время регистрации -> переделаем ее в колонку сколько времения человек уже зарегистрирован

In [ ]:
df2 = pd.DataFrame()
df2 = df1.copy()
df2['timecreated'] = df2['timecreated'] -1681309796
df2.head()

,component,action,target,userid,courseid,ip,timecreated,origin
0,core,graded,user,42067,8580,77.73.31.51,1,web
1,mod_assign,graded,submission,42067,8580,77.73.31.51,1,web
2,mod_quiz,viewed,attempt,64913,4436,45.147.80.190,1,web
3,core,viewed,course,66783,2616,213.87.158.247,1,web
4,mod_assign,viewed,grading_form,42067,8580,77.73.31.51,1,web


In [ ]:
df2.tail()

,component,action,target,userid,courseid,ip,timecreated,origin
47370,core,loggedin,user,66506,0,212.16.10.193,13930,web
47371,core,viewed,dashboard,66506,0,212.16.10.193,13931,web
47372,mod_resource,viewed,course_module,20449,9292,195.225.109.79,13931,web
47373,core,viewed,course,58446,2182,109.252.168.38,13932,web
47374,core,failed,webservice_login,0,0,193.41.142.196,13932,ws


Теперь можно сделпть One-Hot, столбцов будет уже меньше

In [ ]:
df2.head()

,component,action,target,userid,courseid,ip,timecreated,origin
0,core,graded,user,42067,8580,77.73.31.51,1,web
1,mod_assign,graded,submission,42067,8580,77.73.31.51,1,web
2,mod_quiz,viewed,attempt,64913,4436,45.147.80.190,1,web
3,core,viewed,course,66783,2616,213.87.158.247,1,web
4,mod_assign,viewed,grading_form,42067,8580,77.73.31.51,1,web


In [ ]:
df_new = pd.get_dummies(data=df2, columns=['component', 'action', 'target', 'origin'])
df_new.head()

,userid,courseid,ip,timecreated,component_assignsubmission_file,component_core,component_gradereport_overview,component_mod_assign,component_mod_folder,component_mod_forum,...,target_user_enrolment,target_user_list,target_user_login,target_user_password,target_user_profile,target_webservice_function,target_webservice_login,target_webservice_token,origin_web,origin_ws
0,42067,8580,77.73.31.51,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,42067,8580,77.73.31.51,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,64913,4436,45.147.80.190,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,66783,2616,213.87.158.247,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,42067,8580,77.73.31.51,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


Проведем стандартизацию текста

In [ ]:
df_new[['ip_1', 'ip_2']] = df_new['ip']. str.split('.', 1 , expand= True )
df_new[['ip_2', 'ip_3']] = df_new['ip_2']. str.split('.', 1 , expand= True )
df_new[['ip_3', 'ip_4']] = df_new['ip_3']. str.split('.', 1 , expand= True )
df_new.head()

<ipython-input-56-9de92fc8a692>:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_new[['ip_1', 'ip_2']] = df_new['ip']. str.split('.', 1 , expand= True )
<ipython-input-56-9de92fc8a692>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_new[['ip_2', 'ip_3']] = df_new['ip_2']. str.split('.', 1 , expand= True )
<ipython-input-56-9de92fc8a692>:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_new[['ip_3', 'ip_4']] = df_new['ip_3']. str.split('.', 1 , expand= True )


,userid,courseid,ip,timecreated,component_assignsubmission_file,component_core,component_gradereport_overview,component_mod_assign,component_mod_folder,component_mod_forum,...,target_user_profile,target_webservice_function,target_webservice_login,target_webservice_token,origin_web,origin_ws,ip_1,ip_2,ip_3,ip_4
0,42067,8580,77.73.31.51,1,0,1,0,0,0,0,...,0,0,0,0,1,0,77,73,31,51
1,42067,8580,77.73.31.51,1,0,0,0,1,0,0,...,0,0,0,0,1,0,77,73,31,51
2,64913,4436,45.147.80.190,1,0,0,0,0,0,0,...,0,0,0,0,1,0,45,147,80,190
3,66783,2616,213.87.158.247,1,0,1,0,0,0,0,...,0,0,0,0,1,0,213,87,158,247
4,42067,8580,77.73.31.51,1,0,0,0,1,0,0,...,0,0,0,0,1,0,77,73,31,51


In [ ]:
df_new = df_new.drop(['ip'], axis=1)

In [ ]:
df_new['ip_1'] = pd.to_numeric(df_new['ip_1'])
df_new['ip_2'] = pd.to_numeric(df_new['ip_2'])
df_new['ip_3'] = pd.to_numeric(df_new['ip_3'])
df_new['ip_4'] = pd.to_numeric(df_new['ip_4'])
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46114 entries, 0 to 47374
Data columns (total 73 columns):
 #   Column                               Non-Null Count  Dtype
---  ------                               --------------  -----
 0   userid                               46114 non-null  int64
 1   courseid                             46114 non-null  int64
 2   timecreated                          46114 non-null  int64
 3   component_assignsubmission_file      46114 non-null  uint8
 4   component_core                       46114 non-null  uint8
 5   component_gradereport_overview       46114 non-null  uint8
 6   component_mod_assign                 46114 non-null  uint8
 7   component_mod_folder                 46114 non-null  uint8
 8   component_mod_forum                  46114 non-null  uint8
 9   component_mod_page                   46114 non-null  uint8
 10  component_mod_quiz                   46114 non-null  uint8
 11  component_mod_resource               46114 non-null  u

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df_kmeans = scaler.fit_transform(df_new)

Теперь кластеризуем данные с таблицы на 3 кластера

In [ ]:
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=3)
clusters = kmeans_model.fit_predict(df_new)
df_new.insert(df_new.columns.get_loc("userid"), "Cluster", clusters)
df_new.head(3)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,Cluster,userid,courseid,timecreated,component_assignsubmission_file,component_core,component_gradereport_overview,component_mod_assign,component_mod_folder,component_mod_forum,...,target_user_profile,target_webservice_function,target_webservice_login,target_webservice_token,origin_web,origin_ws,ip_1,ip_2,ip_3,ip_4
0,2,42067,8580,1,0,1,0,0,0,0,...,0,0,0,0,1,0,77,73,31,51
1,2,42067,8580,1,0,0,0,1,0,0,...,0,0,0,0,1,0,77,73,31,51
2,0,64913,4436,1,0,0,0,0,0,0,...,0,0,0,0,1,0,45,147,80,190


Gaussian mixture